In [118]:
import requests
import json
from pprint import pprint
import numpy as np
import pandas as pd

api_key = "rA0vy4zNu4RpkIseRL4eccSlExzx2RMphwqHRUQo"

# # Pull ORI Codes from original csv

In [138]:
project_one_df = pd.read_csv("data_for_project_one.csv")
project_one_df

,State,State Ab.,State Capital,Population,ORI Code
0,ALABAMA,AL,Montgomery,"198,525",AL0030100
1,ALASKA,AK,Juneau,"32,113",AK0010300
2,ARIZONA,AZ,Phoenix,"1,680,992",AZ0072300
3,ARKANSAS,AR,Little Rock,"197,312",AR0600200
4,CALIFORNIA,CA,Sacramento,"513,624",CA0340400
5,COLORADO,CO,Denver,"727,211",CODPD0000
6,CONNECTICUT,CT,Hartford,"122,105",CT0006400
7,DELAWARE,DE,Dover,"38,079",DE0010100
8,FLORIDA,FL,Tallahassee,"194,500",FL0370300
9,GEORGIA,GA,Atlanta,"506,811",GAAPD0000


In [139]:
#states 21-30
ori = project_one_df["ORI Code"][19:38].tolist()
ori

['MD0020100',
 'MA0130100',
 'MI3351900',
 'MN0620900',
 'MS0250100',
 'MO0260100',
 'MT0250100',
 'NB0550100',
 'NV0130000',
 'NH0071600',
 'NJ0111100',
 'NM0260100',
 'NY0010100',
 'NC0920100',
 'ND0080100',
 'OHCOP0000',
 'OK0550400',
 'OR0240200',
 'PA0220200']

# # Loop through API endpoint for crime case num

In [140]:

begin_year = "2018"
end_year = "2018"

simple_assault_cases = []
stolen_property_cases = []
rape_cases = []
ori_list = ori


for ori2 in range(len(ori_list)):
    response2 = requests.get(f"https://api.usa.gov/crime/fbi/sapi/api/data/arrest/agencies/offense/{ori_list[ori2]}/all/{begin_year}/{end_year}?API_KEY={api_key}").json()
    try:
        simple_assault_cases.append(response2["results"][0]["simple_assault"])
        stolen_property_cases.append(response2["results"][0]["stolen_property"])
        rape_cases.append(response2["results"][0]["rape"])
    except IndexError:
        simple_assault_cases.append(np.nan)
        stolen_property_cases.append(np.nan)
        rape_cases.append(np.nan)
    # Printing queries
    print(f"Processing record {ori2 + 1}")
print("------")
print("Done")
print("------")

Processing record 1
Processing record 2
Processing record 3
Processing record 4
Processing record 5
Processing record 6
Processing record 7
Processing record 8
Processing record 9
Processing record 10
Processing record 11
Processing record 12
Processing record 13
Processing record 14
Processing record 15
Processing record 16
Processing record 17
Processing record 18
Processing record 19
------
Done
------


# # Merge case number data with original csv 

In [149]:
df=pd.DataFrame(list(zip(ori_list, simple_assault_cases, stolen_property_cases, rape_cases)), 
               columns = ['ORI Code', "Simple Assault Cases", "Stolen Property Cases", "Rape Cases"])

df["No. of Offenses"]=df["Simple Assault Cases"]+df["Stolen Property Cases"] +df["Rape Cases"]

cases_merged_df = pd.merge(project_one_df, df, on= "ORI Code")
cases_merged_df.head()

,State,State Ab.,State Capital,Population,ORI Code,Simple Assault Cases,Stolen Property Cases,Rape Cases,No. of Offenses
0,MARYLAND,MD,Annapolis,"39,174",MD0020100,162.0,3.0,5.0,170.0
1,MASSACHUSETTS,MA,Boston,"692,600",MA0130100,1612.0,169.0,43.0,1824.0
2,MICHIGAN,MI,Lansing,"118,210",MI3351900,791.0,22.0,17.0,830.0
3,MINNESOTA,MN,Saint Paul,"308,096",MN0620900,984.0,43.0,16.0,1043.0
4,MISSISSIPPI,MS,Jackson,"160,628",MS0250100,445.0,106.0,8.0,559.0


# # Loop through API endpoint for number of officers per ori 

In [142]:
ori_list = ori
female =[]
male =[]
begin_year = "2018"
end_year = "2018"

print("Beginning Data Retrieval")
print("------------------------")
for ori2 in range(len(ori_list)):
    response = requests.get(f"https://api.usa.gov/crime/fbi/sapi/api/police-employment/agencies/{ori_list[ori2]}/{begin_year}/{end_year}?API_KEY={api_key}").json()

    try:
        print(f"Processing record {ori2 + 1}")
        female.append(response["results"][0]["female_officer_ct"])
        male.append(response["results"][0]["male_officer_ct"])
    except:
        print("Not found")
        pass
print("------------------------")
print("Data Retrieval Complete")
print("------------------------")

Beginning Data Retrieval
------------------------
Processing record 1
Processing record 2
Processing record 3
Processing record 4
Processing record 5
Processing record 6
Processing record 7
Processing record 8
Processing record 9
Processing record 10
Processing record 11
Processing record 12
Processing record 13
Processing record 14
Processing record 15
Processing record 16
Processing record 17
Processing record 18
Processing record 19
------------------------
Data Retrieval Complete
------------------------


In [150]:
officer_df = pd.DataFrame({"ORI Code": ori_list, "Male Officers": male, "Female Officers" : female})
officer_df["No. of Officers"]=officer_df["Male Officers"]+officer_df["Female Officers"]
officer_df.head()

,ORI Code,Male Officers,Female Officers,No. of Officers
0,MD0020100,106.0,13.0,119.0
1,MA0130100,1839.0,283.0,2122.0
2,MI3351900,149.0,39.0,188.0
3,MN0620900,540.0,91.0,631.0
4,MS0250100,NaN,NaN,NaN


In [151]:
officer_merged_df = pd.merge(cases_merged_df, officer_df, on= "ORI Code")
officer_merged_df.head()

,State,State Ab.,State Capital,Population,ORI Code,Simple Assault Cases,Stolen Property Cases,Rape Cases,No. of Offenses,Male Officers,Female Officers,No. of Officers
0,MARYLAND,MD,Annapolis,"39,174",MD0020100,162.0,3.0,5.0,170.0,106.0,13.0,119.0
1,MASSACHUSETTS,MA,Boston,"692,600",MA0130100,1612.0,169.0,43.0,1824.0,1839.0,283.0,2122.0
2,MICHIGAN,MI,Lansing,"118,210",MI3351900,791.0,22.0,17.0,830.0,149.0,39.0,188.0
3,MINNESOTA,MN,Saint Paul,"308,096",MN0620900,984.0,43.0,16.0,1043.0,540.0,91.0,631.0
4,MISSISSIPPI,MS,Jackson,"160,628",MS0250100,445.0,106.0,8.0,559.0,NaN,NaN,NaN


In [152]:
officer_merged_df.to_csv("merged_dfs.csv",
                  encoding="utf-8", index=False, header=True)